In [0]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
def init_weights(shape):
  init_random_dist = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(init_random_dist)

In [0]:
def init_bias(shape):
  init_bias_vals = tf.constant(0.1, shape=shape)
  return tf.Variable(init_bias_vals)

In [0]:
def conv2d(X, W):
  # X --- [batch, H, W, Channels]
  #W ---- [filter H, filter W, channelsInt, channelsOut]
  return tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME')

In [0]:
def max_pool_2by2(X):
  #X --- [batch, height, w, c]
  return tf.nn.max_pool(X, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [0]:
#Covolutional Layer
def convolutional_layer(input_x, shape):
  
  W = init_weights(shape)
  b = init_bias([shape[3]])
  Z = tf.nn.relu(conv2d(input_x, W) + b)
  
  return Z

In [0]:
def normal_fully_connected_layer(input_layer, size):
  input_size = int(input_layer.get_shape()[1])
  W = init_weights([input_size, size])
  b = init_bias([size])
  
  return tf.matmul(input_layer, W) + b

In [0]:
#Placeholders
X = tf.placeholder(tf.float32, shape=[None, 784])
Y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [0]:
X_image = tf.reshape(X, [-1,28,28,1])

In [0]:
convo_1 = convolutional_layer(X_image, shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [0]:
convo_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [0]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1, 7*7*64])
fully_connected = tf.nn.relu(normal_fully_connected_layer(convo_2_flat, 1024))

In [0]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(fully_connected, keep_prob=hold_prob)

y_pred = normal_fully_connected_layer(full_one_dropout, 10)

In [33]:
# LOSS
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_true, logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [0]:
init = tf.global_variables_initializer()

In [0]:
steps = 5000

with tf.Session() as sess:
  sess.run(init)
  
  for i in range(steps):
    batch_x, batch_y = mnist.train.next_batch(100)
    sess.run(train, feed_dict={X:batch_x, Y_true:batch_y, hold_prob:0.49})
    
    if i % 100 == 0:
      print("ON STEP: {}".format(i))
      print("ACCURACY: ")
      
      matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(Y_true, 1))
      acc = tf.reduce_mean(tf.cast(matches, tf.float32))
      
      print(sess.run(acc, feed_dict={X:mnist.test.images, Y_true:mnist.test.labels, hold_prob:1.0}))
      print('\n')
      